In [ ]:
!pip install pandas numpy torch transformers sentence-transformers implicit scipy nltk rouge-score google-generativeai bert_score bitsandbytes

In [2]:
import json
from tqdm import tqdm
import pandas as pd  
import numpy as np

In [3]:
data = []
filename = '/kaggle/input/regen-reviews-enhanced-with-generative-narratives/regen/REGEN/office.jsonl'
unique_reviewer_ids = set()

with open(filename, 'r') as f:
    for line in tqdm(f, desc="Processing dataset", unit="lines"):
        user = json.loads(line)
        reviewer_id = user.get("reviewer_id")
        
        if reviewer_id not in unique_reviewer_ids:
            if len(unique_reviewer_ids) < 10000:
                unique_reviewer_ids.add(reviewer_id)
            else:
                break

        purchase_history = user.get("purchase_history", [])
        user_summary = user.get('user', None)
        last_purchase = user.get('last_purchase', None)
        
        for purchase in purchase_history:
            item = purchase.get("item", {})
            review = purchase.get("review", {})
            
            row = {
                "reviewer_id": reviewer_id,
                "asin": item.get("asin"),
                "title": item.get("title"),
                "category": item.get("category"),
                "price": item.get("price"),
                "summary": review.get("summary"),
                "rating": review.get("rating"),
                "review": review.get("text"),
                "unix_time": review.get("unix_time"),
                "user_summary": user_summary,
            }
            data.append(row)
 
df = pd.DataFrame(data)

Processing dataset: 10000lines [00:01, 7124.52lines/s]


In [4]:
total_unique_ids = df['reviewer_id'].nunique()
print("Total number of unique reviewer_ids:", total_unique_ids)

Total number of unique reviewer_ids: 10000


In [5]:
df.head()

,reviewer_id,asin,title,category,price,summary,rating,review,unix_time,user_summary
0,A7W1AR9ITP8CV,B00EKWE55K,"Brother Mobile Color Page Scanner, DS-620, Fas...",Office Products - Office Electronics - Scanner...,6.35,One Star,1,"Didn't work properly, only scanned a few lines...",1469318400,"{'profile': 'Business owner, shipping supplies..."
1,A7W1AR9ITP8CV,B00AI652CM,Pratt Multipurpose Kraft Paper Sheet for Packa...,Office Products - Office & School Supplies - E...,218.18,Five Stars,5,Great value for packing paper. It's quite thin...,1469404800,"{'profile': 'Business owner, shipping supplies..."
2,A7W1AR9ITP8CV,B000GIP0P6,"#2 8.5x12"" POLY BUBBLE MAILERS PADDED ENVELOPE...",Office Products - Office & School Supplies - E...,23.95,Five Stars,5,"Great value, lightweight with nice bubble and ...",1469404800,"{'profile': 'Business owner, shipping supplies..."
3,A7W1AR9ITP8CV,B00DGDQUJS,2 Rolls Of 12-Inch-by-175-Feet Wrap by The Boxery,Office Products - Office & School Supplies - E...,24.99,Great value for bubble wrap,5,Great value for bubble wrap. The quality varie...,1469404800,"{'profile': 'Business owner, shipping supplies..."
4,A7W1AR9ITP8CV,B00P9GT4MA,Generic Bubble Bags Flat Open Top Smooth on Bo...,Office Products - Office & School Supplies - E...,9.74,These are great little bubble bags that work p...,5,These are great little bubble bags that work p...,1469404800,"{'profile': 'Business owner, shipping supplies..."


In [6]:
df['long_summary'] = df['user_summary'].apply(lambda x: x.get('long_summary') if isinstance(x, dict) else None)
df['purchase_date'] = pd.to_datetime(df['unix_time'], unit='s')
df = df.drop(['summary', 'user_summary', 'unix_time'], axis=1)
df['category'] = df['category'].str.split("-", n=1).str[1]

In [7]:
df.head()

,reviewer_id,asin,title,category,price,rating,review,long_summary,purchase_date
0,A7W1AR9ITP8CV,B00EKWE55K,"Brother Mobile Color Page Scanner, DS-620, Fas...",Office Electronics - Scanners & Accessories -...,6.35,1,"Didn't work properly, only scanned a few lines...",This user has a strong focus on office and shi...,2016-07-24
1,A7W1AR9ITP8CV,B00AI652CM,Pratt Multipurpose Kraft Paper Sheet for Packa...,"Office & School Supplies - Envelopes, Mailers...",218.18,5,Great value for packing paper. It's quite thin...,This user has a strong focus on office and shi...,2016-07-25
2,A7W1AR9ITP8CV,B000GIP0P6,"#2 8.5x12"" POLY BUBBLE MAILERS PADDED ENVELOPE...","Office & School Supplies - Envelopes, Mailers...",23.95,5,"Great value, lightweight with nice bubble and ...",This user has a strong focus on office and shi...,2016-07-25
3,A7W1AR9ITP8CV,B00DGDQUJS,2 Rolls Of 12-Inch-by-175-Feet Wrap by The Boxery,"Office & School Supplies - Envelopes, Mailers...",24.99,5,Great value for bubble wrap. The quality varie...,This user has a strong focus on office and shi...,2016-07-25
4,A7W1AR9ITP8CV,B00P9GT4MA,Generic Bubble Bags Flat Open Top Smooth on Bo...,"Office & School Supplies - Envelopes, Mailers...",9.74,5,These are great little bubble bags that work p...,This user has a strong focus on office and shi...,2016-07-25


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76175 entries, 0 to 76174
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   reviewer_id    76175 non-null  object        
 1   asin           76175 non-null  object        
 2   title          76175 non-null  object        
 3   category       74721 non-null  object        
 4   price          76175 non-null  float64       
 5   rating         76175 non-null  int64         
 6   review         76175 non-null  object        
 7   long_summary   76175 non-null  object        
 8   purchase_date  76175 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(6)
memory usage: 5.2+ MB


In [9]:
df.to_csv('REGEN_preprocessed_office.csv', index=False)

**Sample Generation**

In [18]:
import os
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import random
import nltk
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from bert_score import score as bert_score
from sentence_transformers import SentenceTransformer
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

# ----------------------------
# Define fixed prompt for testing
# ----------------------------
fixed_prompt = (
    "As a customer analyst, synthesize a user narrative in about 200 words from this customer's purchase history, reviews, and interaction data. "
    "Focus on identifying statistically significant patterns and trends that reveal their core preferences—especially in colors, brands, sizes, and styles—while also capturing broader behavioral signals across the purchase history. \n"
    "Your narrative should include:\n"
    " - Group related products to expose recurring patterns and underlying purposes of purchases\n"
    " - Highlight seasonal or cyclical buying behaviors\n"
    " - Analyze purchase frequency, order sizes, and brand loyalties\n"
    " - Detail category preferences, including any product avoidances\n"
    " - Evaluate price range preferences\n"
    " - Pinpoint preferences in color, size, and style across different categories\n"
    " - Extract common themes, including satisfaction drivers and pain points\n"
    "\nDeliverable Format:\n"
    "- Present your analysis as a single, focused paragraph.\n"
    "- Emphasize overarching patterns and insights rather than isolated examples.\n"
    "- Use specific examples only when they clearly illustrate significant trends (e.g., key colors, brands, sizes).\n"
    "- Conclude with the customer's overall profile.\n"
    "\nGuidelines:\n"
    "- Cover all purchase history.\n"
    "- Maintain analytical objectivity and clarity.\n"
    "- Avoid extraneous commentary that is not part of the narrative."
)

# ----------------------------
# RE-DEFINE NECESSARY FUNCTIONS & CLASSES
# ----------------------------

def preprocess_dataset(df):
    expected_columns = ['title', 'category', 'price', 'reviewer_id', 'asin', 'rating', 'review', 'purchase_date']
    for col in expected_columns:
        assert col in df.columns, f"DataFrame is missing expected column: {col}"
    df.fillna("", inplace=True)
    df['purchase_date'] = pd.to_datetime(df['purchase_date'], errors='coerce')
    
    # Mark the final purchase for each reviewer using your original logic:
    df['final_item'] = False
    for user, group in df.groupby('reviewer_id'):
        max_date = group['purchase_date'].max()
        group_max = group[group['purchase_date'] == max_date]
        final_idx = group_max.index[-1]
        df.at[final_idx, 'final_item'] = True

    # Create metadata column
    df['metadata'] = (df['title'] + " " + df['category'] + " Price: " +
                      df['price'].astype(str) + " Date: " +
                      df['purchase_date'].astype(str) + " Final Item: " +
                      df['final_item'].astype(str))
    
    # --- Reorder Purchases ---
    # For each reviewer, bring the final purchase (final_item == True) to the top,
    # then sort the remaining purchases by purchase_date descending (most recent to oldest)
    def reorder_group(group):
        return group.sort_values(by=['final_item', 'purchase_date'], ascending=[False, False])
    
    df = df.groupby('reviewer_id', group_keys=False).apply(reorder_group).reset_index(drop=True)
    
    # Build user interactions dictionary (ordered as in the DataFrame)
    user_interactions = {}
    for _, row in df.iterrows():
        user_id = row['reviewer_id']
        asin = row['asin']
        try:
            rating = float(row['rating'])
        except Exception as e:
            raise ValueError(f"Error converting rating to float: {e}")
        review = row['review']
        metadata = row['metadata']
        interaction = (asin, metadata, rating, review)
        user_interactions.setdefault(user_id, []).append(interaction)
    
    user_ids = list(user_interactions.keys())
    user2idx = {u: i for i, u in enumerate(user_ids)}
    item_ids = df['asin'].unique().tolist()
    item2idx = {asin: i for i, asin in enumerate(item_ids)}
    return user_interactions, user2idx, item2idx, df

# (2) CF Encoder class (must match training definition)
class TrainableCFEncoder(nn.Module):
    def __init__(self, num_items, cf_dim=128):
        super(TrainableCFEncoder, self).__init__()
        self.item_embeddings = nn.Embedding(num_items, cf_dim)
        nn.init.xavier_uniform_(self.item_embeddings.weight)
        
    def forward(self, item_indices):
        return self.item_embeddings(item_indices)

# (3) Item Encoder (for unseen items)
class ItemEncoder(nn.Module):
    def __init__(self, input_dim, cf_dim):
        super(ItemEncoder, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, cf_dim)
        )
        
    def forward(self, x):
        return self.fc(x)

# (4) Advanced Fusion Adapter
class AdvancedFusionAdapter(nn.Module):
    def __init__(self, cf_dim, semantic_dim, model_dim=1024, num_layers=2, num_heads=4, dropout=0.1):
        super(AdvancedFusionAdapter, self).__init__()
        self.input_proj_cf = nn.Linear(cf_dim, model_dim)
        self.input_proj_semantic = nn.Linear(semantic_dim, model_dim)
        self.layers = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout)
            for _ in range(num_layers)
        ])
        
    def forward(self, cf_emb, semantic_emb):
        cf_emb = cf_emb.unsqueeze(1)
        semantic_emb = semantic_emb.unsqueeze(1)
        cf_proj = self.input_proj_cf(cf_emb)
        sem_proj = self.input_proj_semantic(semantic_emb)
        fused = cf_proj + sem_proj
        for layer in self.layers:
            fused = layer(fused)
        return fused.squeeze(1)

# (5) Cross-Attention Block & Soft Prompt Generator
class CrossAttentionBlock(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.1):
        super(CrossAttentionBlock, self).__init__()
        self.cross_attn = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads, dropout=dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.ReLU(),
            nn.Linear(d_model * 4, d_model)
        )
        self.norm2 = nn.LayerNorm(d_model)
        
    def forward(self, query, key, value):
        attn_output, _ = self.cross_attn(query, key, value)
        query = self.norm1(query + attn_output)
        ff_output = self.ff(query)
        out = self.norm2(query + ff_output)
        return out

class SoftPromptGenerator(nn.Module):
    def __init__(self, num_soft_tokens, d_model, num_heads, num_layers, dropout=0.1):
        super(SoftPromptGenerator, self).__init__()
        self.soft_tokens = nn.Parameter(torch.randn(num_soft_tokens, d_model))
        self.blocks = nn.ModuleList([
            CrossAttentionBlock(d_model, num_heads, dropout)
            for _ in range(num_layers)
        ])
        
    def forward(self, fused_embeddings):
        batch_size = fused_embeddings.size(0)
        soft_prompts = self.soft_tokens.unsqueeze(1).expand(-1, batch_size, -1)
        fused_emb_exp = fused_embeddings.unsqueeze(0)
        for block in self.blocks:
            soft_prompts = block(soft_prompts, fused_emb_exp, fused_emb_exp)
        return soft_prompts

# (6) Helper function for semantic embeddings extraction
def get_semantic_embeddings(texts, semantic_model, device, batch_size=16):
    embeddings_list = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i : i + batch_size]
        batch_features = semantic_model.tokenize(batch_texts)
        batch_features = {k: v.to(device) for k, v in batch_features.items()}
        with torch.no_grad():
            batch_output = semantic_model(batch_features)
        embeddings_list.append(batch_output["sentence_embedding"])
    return torch.cat(embeddings_list, dim=0)

# (7) Get embeddings for a given user
def get_embeddings_for_user(user_interactions, user_id, item2idx, cf_encoder, semantic_model, item_encoder, device):
    interactions = user_interactions[user_id]
    cf_emb_list = []
    texts = []
    for interaction in interactions:
        asin, metadata, rating, review = interaction
        texts.append(metadata + " " + review)
        if asin in item2idx:
            idx = item2idx[asin]
            idx_tensor = torch.tensor([idx], dtype=torch.long, device=device)
            cf_emb = cf_encoder(idx_tensor)
        else:
            meta_emb = get_semantic_embeddings([metadata], semantic_model, device, batch_size=1)
            cf_emb = item_encoder(meta_emb)
        cf_emb_list.append(cf_emb)
    cf_emb_tensor = torch.cat(cf_emb_list, dim=0)
    semantic_emb_tensor = get_semantic_embeddings(texts, semantic_model, device, batch_size=16)
    return cf_emb_tensor, semantic_emb_tensor

# (8) Generate an interaction summary for a user
def generate_interaction_summary(user_interactions, user_id):
    interactions = user_interactions[user_id]
    summary_lines = []
    for asin, metadata, rating, review in interactions:
        summary_lines.append(f"Item: {asin}. Metadata: {metadata}. Rating: {rating}. Review: {review}.")
    return "\n".join(summary_lines)

# (9) Batched LLM input preparation function
def prepare_llm_inputs(soft_prompts, prompt_text, tokenizer, llm, device, max_length=512):
    inputs = tokenizer(prompt_text, return_tensors="pt", max_length=max_length, truncation=True).to(device)
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    input_embeds = llm.encoder.embed_tokens(input_ids)
    # soft_prompts: (num_soft_tokens, batch, d_model) -> (batch, num_soft_tokens, d_model)
    soft_prompt_embeds = soft_prompts.squeeze(1).unsqueeze(0)
    inputs_embeds = torch.cat([soft_prompt_embeds, input_embeds], dim=1)
    soft_prompt_attention = torch.ones((1, soft_prompt_embeds.size(1)), device=device)
    new_attention_mask = torch.cat([soft_prompt_attention, attention_mask], dim=1)
    return inputs_embeds, new_attention_mask

# (10) Function to generate narratives in batch
def generate_narrative_with_soft_prompt_batch(soft_prompts_batch, prompt_texts, tokenizer, llm, device,
                                              max_output_tokens=150, num_beams=5, temperature=1.0):
    inputs_embeds, new_attention_mask = prepare_llm_inputs_batch(soft_prompts_batch, prompt_texts, tokenizer, llm, device)
    num_soft_tokens = soft_prompts_batch.size(0)
    max_length = num_soft_tokens + max_output_tokens + 150
    generated_ids = llm.generate(
         inputs_embeds=inputs_embeds,
         attention_mask=new_attention_mask,
         max_length=max_length,
         repetition_penalty=1.2,
         temperature=temperature,
         no_repeat_ngram_size=3,
         decoder_start_token_id=llm.config.decoder_start_token_id,
         num_beams=num_beams,
         num_return_sequences=1
    )
    narratives = []
    for gen_ids in generated_ids:
        text = tokenizer.decode(gen_ids, skip_special_tokens=True)
        text = text.lstrip(" ,")
        if text and not text[0].isupper():
            text = text[0].upper() + text[1:]
        narratives.append(text)
    return narratives

# (11) Evaluation function for narratives
def evaluate_narrative(generated, reference):
    # If the reference is empty, return None to signal skipping evaluation.
    if not reference.strip():
        return None
    generated_tokens = word_tokenize(generated.lower())
    reference_tokens = [word_tokenize(reference.lower())]
    bleu_score = sentence_bleu(reference_tokens, generated_tokens)
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(reference, generated)
    P, R, F1 = bert_score([generated], [reference], lang="en", verbose=False)
    bert_f1 = F1.item()
    return bleu_score, rouge_scores, bert_f1

# (12) Build a reference map from the test dataframe
def build_user_reference_map(df):
    user_reference = {}
    for _, row in df.iterrows():
        user_id = row['reviewer_id']
        long_summary = row.get('long_summary', "")
        if user_id not in user_reference and long_summary:
            user_reference[user_id] = long_summary
    return user_reference

# ----------------------------
# LOAD SAVED MODELS & ITEMS
# ----------------------------

with open('/kaggle/input/final-long-office/item2idx.pkl', 'rb') as f:
    item2idx = pickle.load(f)
cf_dim = 128
num_items = len(item2idx)
cf_encoder = TrainableCFEncoder(num_items, cf_dim=cf_dim).to(device)
cf_encoder.load_state_dict(torch.load('/kaggle/input/final-long-office/trainable_cf_encoder.pth', map_location=device))
cf_encoder.eval()
semantic_model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
semantic_model.to(device)
semantic_dim = semantic_model.get_sentence_embedding_dimension()
item_encoder = ItemEncoder(semantic_dim, cf_dim).to(device)
fusion_adapter = AdvancedFusionAdapter(cf_dim=cf_dim, semantic_dim=semantic_dim,
                                         model_dim=1024, num_layers=2, num_heads=4, dropout=0.1).to(device)
fusion_adapter.load_state_dict(torch.load('/kaggle/input/final-long-office/fusion_adapter_model.pth', map_location=device))
fusion_adapter.eval()
soft_prompt_generator = SoftPromptGenerator(num_soft_tokens=20, d_model=1024, num_heads=4, num_layers=2, dropout=0.1).to(device)
soft_prompt_generator.load_state_dict(torch.load('/kaggle/input/final-long-office/soft_prompt_generator.pth', map_location=device))
soft_prompt_generator.eval()
model_name = "google/flan-t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
llm = T5ForConditionalGeneration.from_pretrained('/kaggle/input/final-long-office/fine_tuned_llm_peft').to(device)
llm.eval()


# ----------------------------
# LOAD TEST DATASET
# ----------------------------
# Assumes a CSV file 'df_test.csv' exists with necessary columns.
df_train, df_temp = train_test_split(df, test_size=0.3, shuffle=False)
df_val, df_test = train_test_split(df_temp, test_size=0.5, shuffle=False)
user_interactions_test, _, _, df_test = preprocess_dataset(df_test)
user_reference_map_test = build_user_reference_map(df_test)


# ----------------------------
# BATCHED TESTING PIPELINE
# ----------------------------
sample_user_id = list(user_interactions_test.keys())[111]
print(f"\nGenerating narrative for sample user: {sample_user_id}")

# Get embeddings and compute soft prompt for the sample user
cf_emb, semantic_emb = get_embeddings_for_user(user_interactions_test, sample_user_id, item2idx,
                                                 cf_encoder, semantic_model, item_encoder, device)
fused_embeddings = fusion_adapter(cf_emb, semantic_emb)
user_fused = fused_embeddings.mean(dim=0, keepdim=True)
soft_prompts = soft_prompt_generator(user_fused)
# soft_prompts: (num_soft_tokens, 1, model_dim) -> add batch dimension as needed
soft_prompts_tensor = soft_prompts.unsqueeze(0).transpose(0, 1)

# Prepare full prompt: fixed prompt + interaction summary (which now reflects the new ordering)
interaction_summary = generate_interaction_summary(user_interactions_test, sample_user_id)
full_prompt = fixed_prompt + "\n\n" + interaction_summary

# Generate narrative for the sample user
inputs_embeds, new_attention_mask = prepare_llm_inputs(soft_prompts, full_prompt, tokenizer, llm, device)
generated_ids = llm.generate(
    inputs_embeds=inputs_embeds,
    attention_mask=new_attention_mask,
    max_length=soft_prompts.size(0) + 200,
    min_length=150,
    repetition_penalty=1.5,
    temperature=1.0,
    no_repeat_ngram_size=3,
    decoder_start_token_id=llm.config.decoder_start_token_id,
    num_beams=5,
    num_return_sequences=1
)
narrative = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print("\n--- Generated Narrative ---")
print(narrative)

# Optionally, if you have reference narrative evaluate the generated narrative.
reference = build_user_reference_map(df_test).get(sample_user_id, "No reference explanation available.")
print("\n--- Reference Narrative ---")
print(reference)

if reference.strip() and reference != "No reference narrative available.":
    bleu, rouge, bert_f1 = evaluate_narrative(narrative, reference)
    print("\n--- Evaluation Metrics ---")
    print(f"BLEU: {bleu:.4f}")
    print(f"ROUGE: {rouge}")
    print(f"BERTScore F1: {bert_f1:.4f}")
else:
    print("\nNo reference narrative available for evaluation.")


Using device: cuda

Generating narrative for sample user: A1AJPSBEMJEPEM

--- Generated Narrative ---
This customer's purchase history reveals a strong interest in office supplies, particularly those related to printing and organization. They have purchased multiple HP printer ink cartridges, including black, tri-color, and multi-packs, indicating a consistent need for these items. Additionally, the customer has purchased a paper shredder, suggesting a need for document shredding in a home office setting. The customer seems to value products that are reliable and functional, as evidenced by their positive reviews of products that meet their expectations. The final purchase of a 4-drawer medium storage cart further supports their focus on organization and storage solutions within an office or home office environment. They seem to be satisfied with the quality and functionality of the products they purchase.

--- Reference Narrative ---
The user appears to be focused on maintaining an or

In [ ]:
filtered_rows = df_test[df_test['reviewer_id'] == 'A1AJPSBEMJEPEM']

for idx, row in filtered_rows.iterrows():
    print(f"Row {idx}:")
    for col in df_test.columns:
        print(f"{col}: {row[col]}")
    print("-" * 40)